In [1]:
%load_ext nbtest

The Jupyter kernel is a nightmare. Having spent a day trying to figure out how to `await` the task that's created when I run an `async` test, I found this bug:

> https://github.com/ipython/ipykernel/pull/589

Any attempt to `await` the runner task will deadlock the cell that's doing the waiting because the way the Jupyter kernel handles widget I/O is terrible. The bug was filed in 2021 and it's not likely to get fixed anytime soon. This project will need a major overhaul if/when they ever get the `async` support right. 

**It's effectively impossible to make a unit test that checks if async tests passed or failed.**

In [2]:
import time
import ipywidgets
from IPython.display import HTML

In [3]:
"""@solution"""

def make_html(name):
    time.sleep(1)
    return HTML(f"Hello <b>{name}</b>!")


In [4]:
"""@widgets"""

display(ipywidgets.interactive(
    make_html,
    {'auto_display': True, 'manual': True},
    name = "Zelda Pinwheel",
))

interactive(children=(Text(value='Zelda Pinwheel', continuous_update=False, description='name'), Button(descri…

In [5]:
%%testing @solution @widgets make_html

import ipywidgets
import asyncio
from contextlib import asynccontextmanager
from IPython.display import HTML 
from unittest import IsolatedAsyncioTestCase, TestCase

def test_solution():
    assert solution 
    assert "make_html" in solution.functions

def test_make_html():
    assert make_html
    assert callable(make_html)
    html = make_html('foo')
    assert isinstance(html, HTML)

@asynccontextmanager
async def wait_for(widget, names):
    fut = asyncio.Future()
    def obs(change):
        fut.set_result(change)
        widget.unobserve(obs, names)
    widget.observe(obs, names=names)
    try:
        yield fut 
    finally:
        await fut 

class WidgetTest(TestCase):

    async def test_widgets(self):
        r = widgets.run()
        name = r.outputs[0].children[0]
        button = r.outputs[0].children[1]
        output = r.outputs[0].children[2]      

        assert isinstance(button, ipywidgets.Button)  
        assert isinstance(output, ipywidgets.Output)  

        name.value = "test person"
        async with wait_for(output, "outputs"):
            button.click()

        assert "test person" in output.outputs[0]['data']['text/html']


Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': 'HTML(value=\'<div style="font-size: la…

In [6]:
%%testing @widgets

async def test_async_function():
    r = widgets.run()
    name = r.outputs[0].children[0]
    button = r.outputs[0].children[1]
    output = r.outputs[0].children[2]      

    assert isinstance(button, ipywidgets.Button)  
    assert isinstance(output, ipywidgets.Output)  

    name.value = "test person"
    async with wait_for(output, "outputs"):
        button.click()

    assert "test person" in output.outputs[0]['data']['text/html']

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': 'HTML(value=\'<div style="font-size: la…